# Functions to use later

In [2]:
pip install pandas-datareader

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas_datareader.data as web
import datetime
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2022,4,19)
end = datetime.datetime(2022,5,19)

In [5]:
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 

In [6]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2022,4,19)
end = datetime.datetime(2022,5,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [7]:
pip install PyPortfolioOpt

Note: you may need to restart the kernel to use updated packages.


In [8]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

In [9]:
from pypfopt.efficient_frontier import EfficientFrontier

In [10]:
import numpy as np

# DIA Stock Discovery

Just get rid of each stock and evaluate their changed performance

In [11]:
# call the function and generate a table with stock adjusted close prices
# Function for computing portfolio return
def portfolio_returns(df,weights,time):
    return (np.sum(df.mean() * weights)) * time
# Function for computing standard deviation of portfolio returns
def portfolio_sd(df,weights,time):
    return np.sqrt(np.transpose(weights) @ (df.cov() * time) @ weights)
def sharpe_fun(df,weights,time,rf):
    return ((portfolio_returns(df,weights,time)-rf) / portfolio_sd(df,weights,time))

def getrid(stocks,time1):
    results_return=pd.DataFrame(columns=stocks)
    #results_volatility=pd.DataFrame(columns=stocks)
    #results_sharpe=pd.DataFrame(columns=stocks)
    for i in range(len(stocks)):
        stock=stocks[0:i]+stocks[i+1:]
        portfolio=combine_stocks(stock)
        # calculate the expected return using historical data
        mu = mean_historical_return(portfolio, returns_data=False)
        # calculate the covariance matrix
        S = CovarianceShrinkage(portfolio).ledoit_wolf()
        from pypfopt.efficient_frontier import EfficientFrontier
        # generate the efficient frontier
        ef = EfficientFrontier(mu,S)
        # maximize sharpe ratio to help us get the portfolio weights
        try:
            weights = ef.max_sharpe()
            break
        except:
            weights=0
            
        if weights!=0:
            cleaned_weights=list(cleaned_weights.values())
            returns=portfolio_returns(df,cleaned_weights,time1)
            sd=portfolio_sd(df,cleaned_weights,time1)
            sharpe=sharpe_fun(df,cleaned_weights,time1,0.0298)
            result=[returns,sd,sharpe]
            results_return[results_return.columns[1]]=np.array(result) 
        else:
            results_return[results_return.columns[i]]=[0,0,0]
    return results_return
        
        

In [11]:
# call the function and generate a table with stock adjusted close prices
# Function for computing portfolio return
def portfolio_returns(df,weights,time):
    return (np.sum(df.mean() * weights)) * time
# Function for computing standard deviation of portfolio returns
def portfolio_sd(df,weights,time):
    return np.sqrt(np.transpose(weights) @ (df.cov() * time) @ weights)
def sharpe_fun(df,weights,time,rf):
    return ((portfolio_returns(df,weights,time)-rf) / portfolio_sd(df,weights,time))

def getrid(stocks,time1):
    results_return=pd.DataFrame(columns=stocks)
    #results_volatility=pd.DataFrame(columns=stocks)
    #results_sharpe=pd.DataFrame(columns=stocks)
    for i in range(len(stocks)):
        stock=stocks[0:i]+stocks[i+1:]
        portfolio=combine_stocks(stock)
        # calculate the expected return using historical data
        mu = mean_historical_return(portfolio, returns_data=False)
        # calculate the covariance matrix
        S = CovarianceShrinkage(portfolio).ledoit_wolf()
        from pypfopt.efficient_frontier import EfficientFrontier
        # generate the efficient frontier
        ef = EfficientFrontier(mu,S)
        # maximize sharpe ratio to help us get the portfolio weights
        weights = ef.max_sharpe()
            
        cleaned_weights=list(cleaned_weights.values())
        returns=portfolio_returns(df,cleaned_weights,time1)
        sd=portfolio_sd(df,cleaned_weights,time1)
        sharpe=sharpe_fun(df,cleaned_weights,time1,0.0298)
        result=[returns,sd,sharpe]
        results_return[results_return.columns[1]]=np.array(result) 
    return results_return
        
        

In [12]:
stocks=["UNH", "GS", "HD","MSFT"]
getrid(stocks,20)

                    GS
Date                  
2022-04-19  331.403107
2022-04-20  336.443939
2022-04-21  329.775391
2022-04-22  315.442047
2022-04-25  317.069763
                    HD
Date                  
2022-04-19  303.885498
2022-04-20  311.132202
2022-04-21  306.412964
2022-04-22  296.293335
2022-04-25  301.061920
                  MSFT
Date                  
2022-04-19  283.235962
2022-04-20  284.288269
2022-04-21  278.778412
2022-04-22  272.047485
2022-04-25  278.689087
                   UNH
Date                  
2022-04-19  534.113403
2022-04-20  542.367920
2022-04-21  533.855103
2022-04-22  517.465149
2022-04-25  520.772949
                    HD
Date                  
2022-04-19  303.885529
2022-04-20  311.132202
2022-04-21  306.412964
2022-04-22  296.293304
2022-04-25  301.061890
                  MSFT
Date                  
2022-04-19  283.235962
2022-04-20  284.288269
2022-04-21  278.778412
2022-04-22  272.047485
2022-04-25  278.689087
                   UNH
Date       

,UNH,GS,HD,MSFT
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0


In [13]:
stocks = ["GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
portfolio=combine_stocks(stocks)
# calculate the expected return using historical data
mu = mean_historical_return(portfolio, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio).ledoit_wolf()
from pypfopt.efficient_frontier import EfficientFrontier
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()

                    GS
Date                  
2022-04-19  331.403107
2022-04-20  336.443939
2022-04-21  329.775391
2022-04-22  315.442047
2022-04-25  317.069763
                    HD
Date                  
2022-04-19  303.885529
2022-04-20  311.132202
2022-04-21  306.412964
2022-04-22  296.293304
2022-04-25  301.061890
                  MSFT
Date                  
2022-04-19  283.235931
2022-04-20  284.288269
2022-04-21  278.778412
2022-04-22  272.047516
2022-04-25  278.689087
                   MCD
Date                  
2022-04-19  252.609589
2022-04-20  253.054688
2022-04-21  252.461243
2022-04-22  247.436737
2022-04-25  250.117126
                  AMGN
Date                  
2022-04-19  247.718094
2022-04-20  249.683289
2022-04-21  249.761490
2022-04-22  244.765488
2022-04-25  246.544876
                   CAT
Date                  
2022-04-19  227.928680
2022-04-20  230.795486
2022-04-21  228.331223
2022-04-22  213.376144
2022-04-25  213.770737
                     V
Date       

In [14]:
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
table=getrid(stocks,20)

                    GS
Date                  
2022-04-19  331.403107
2022-04-20  336.443939
2022-04-21  329.775391
2022-04-22  315.442047
2022-04-25  317.069763
                    HD
Date                  
2022-04-19  303.885498
2022-04-20  311.132202
2022-04-21  306.412964
2022-04-22  296.293335
2022-04-25  301.061920
                  MSFT
Date                  
2022-04-19  283.235962
2022-04-20  284.288269
2022-04-21  278.778412
2022-04-22  272.047485
2022-04-25  278.689087
                   MCD
Date                  
2022-04-19  252.609589
2022-04-20  253.054688
2022-04-21  252.461243
2022-04-22  247.436737
2022-04-25  250.117126
                  AMGN
Date                  
2022-04-19  247.718109
2022-04-20  249.683289
2022-04-21  249.761505
2022-04-22  244.765488
2022-04-25  246.544891
                   CAT
Date                  
2022-04-19  227.928696
2022-04-20  230.795471
2022-04-21  228.331223
2022-04-22  213.376144
2022-04-25  213.770737
                     V
Date       

In [16]:
results_return=pd.DataFrame(columns=stocks)

In [17]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=stocks[1:]
stock=stocks[0:23]+stocks[24:]
portfolio=combine_stocks(stock)  
# calculate the expected return using historical data
mu = mean_historical_return(portfolio, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=list(cleaned_weights.values())
returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
results_return[results_return.columns[23]]=np.array(result) 

                   UNH
Date                  
2022-04-19  534.113403
2022-04-20  542.367920
2022-04-21  533.855103
2022-04-22  517.465149
2022-04-25  520.772949
                    GS
Date                  
2022-04-19  331.403076
2022-04-20  336.443909
2022-04-21  329.775391
2022-04-22  315.442047
2022-04-25  317.069733
                    HD
Date                  
2022-04-19  303.885498
2022-04-20  311.132202
2022-04-21  306.412964
2022-04-22  296.293335
2022-04-25  301.061920
                  MSFT
Date                  
2022-04-19  283.235931
2022-04-20  284.288269
2022-04-21  278.778412
2022-04-22  272.047516
2022-04-25  278.689087
                   MCD
Date                  
2022-04-19  252.609589
2022-04-20  253.054688
2022-04-21  252.461243
2022-04-22  247.436737
2022-04-25  250.117126
                  AMGN
Date                  
2022-04-19  247.718109
2022-04-20  249.683273
2022-04-21  249.761505
2022-04-22  244.765488
2022-04-25  246.544891
                   CAT
Date       

In [106]:
results_return.to_csv("DIAdeletemean.csv", index=True)

Replace one of the stock with 10 year treasure

In [109]:
results_return2=pd.DataFrame(columns=stocks)

In [156]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=['^TNX']+stocks[1:]
stock=stocks[0:29]+['^TNX']+stocks[30:]
portfolio=combine_stocks(stock)  
# calculate the expected return using historical data
mu = mean_historical_return(portfolio, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=list(cleaned_weights.values())
returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
results_return2[results_return2.columns[29]]=np.array(result) 

                   UNH
Date                  
2022-04-19  537.700012
2022-04-20  546.010010
2022-04-21  537.440002
2022-04-22  520.940002
2022-04-25  524.270020
                    GS
Date                  
2022-04-19  333.905151
2022-04-20  338.984039
2022-04-21  332.265167
2022-04-22  317.823608
2022-04-25  319.463593
                    HD
Date                  
2022-04-19  307.799988
2022-04-20  315.140015
2022-04-21  310.359985
2022-04-22  300.109985
2022-04-25  304.940002
                  MSFT
Date                  
2022-04-19  285.299988
2022-04-20  286.359985
2022-04-21  280.809998
2022-04-22  274.029999
2022-04-25  280.720001
                   MCD
Date                  
2022-04-19  255.399994
2022-04-20  255.850006
2022-04-21  255.250000
2022-04-22  250.169998
2022-04-25  252.880005
                  AMGN
Date                  
2022-04-19  253.369995
2022-04-20  255.380005
2022-04-21  255.460007
2022-04-22  250.350006
2022-04-25  252.169998
                   CAT
Date       

In [157]:
results_return2

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.063701,0.063656,0.063701,0.063648,0.063701,0.063701,0.063701,0.063701,0.063701,0.063701,0.063701,0.063701,0.063701,0.063672,0.063701,0.063612,0.063701,0.063701,0.063701,0.063701,0.063701,0.063701,0.063701,0.024591,0.063701,0.063701,0.063701,0.063701,0.063701,0.063657
1,0.064929,0.064886,0.064929,0.064877,0.064929,0.064929,0.064929,0.064929,0.064929,0.064929,0.064929,0.064929,0.064929,0.064901,0.064929,0.064842,0.064929,0.064929,0.064929,0.064929,0.064929,0.064929,0.064929,0.075022,0.064929,0.064929,0.064929,0.064929,0.064929,0.064887
2,0.522119,0.521785,0.522119,0.521724,0.522119,0.522119,0.522119,0.522119,0.522119,0.522119,0.522119,0.522119,0.522119,0.521905,0.522119,0.521456,0.522119,0.522119,0.522119,0.522119,0.522119,0.522119,0.522119,-0.069435,0.522119,0.522119,0.522119,0.522119,0.522119,0.521794


In [158]:
results_return2.to_csv("DIAswapmeanmonth.csv", index=True)

In [123]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=stocks[1:]
stock=stocks[0:2]+stocks[3:]
portfolio=combine_stocks(stock)  
t=len(portfolio.iloc[0:])
num=[]
for i in range(t):
    num.append(1)
    
portfolio['extra']=num
# calculate the expected return using historical data
mu = mean_historical_return(portfolio, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=list(cleaned_weights.values())
returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
#results_return2[results_return2.columns[2]]=np.array(result) 

                   UNH
Date                  
2022-04-19  535.789307
2022-04-20  544.069763
2022-04-21  535.530212
2022-04-22  519.088867
2022-04-25  522.407043
                    GS
Date                  
2022-04-19  333.905151
2022-04-20  338.984039
2022-04-21  332.265167
2022-04-22  317.823608
2022-04-25  319.463593
                  MSFT
Date                  
2022-04-19  284.034149
2022-04-20  285.089447
2022-04-21  279.564087
2022-04-22  272.814178
2022-04-25  279.474487
                   MCD
Date                  
2022-04-19  253.992325
2022-04-20  254.439850
2022-04-21  253.843155
2022-04-22  248.791153
2022-04-25  251.486221
                  AMGN
Date                  
2022-04-19  249.424332
2022-04-20  251.403030
2022-04-21  251.481796
2022-04-22  246.451370
2022-04-25  248.243027
                   CAT
Date                  
2022-04-19  229.453812
2022-04-20  232.339783
2022-04-21  229.859039
2022-04-22  214.803894
2022-04-25  215.201126
                     V
Date       

Test risk parity portfolio while deleting each stock

In [165]:
from __future__ import division
import numpy as np
from matplotlib import pyplot as plt
from numpy.linalg import inv,pinv
from scipy.optimize import minimize


import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
from scipy.optimize import minimize
TOLERANCE = 1e-10


def _allocation_risk(weights, covariances):

    # We calculate the portfolio risk (sigma^2)
    portfolio_risk = np.sqrt((weights * covariances * weights.T))[0, 0]  

    # It returns the risk of the weights distribution
    return portfolio_risk


def _assets_risk_contribution_to_allocation_risk(weights, covariances):

    # We calculate the risk of the portfolio
    portfolio_risk = _allocation_risk(weights, covariances)

    # We calculate the contribution of each asset to the risk of total portfolio risk
    assets_risk_contribution = np.multiply(weights.T, covariances * weights.T)/portfolio_risk

    # It returns the contribution of each asset to the risk of the weights
    # distribution
    return assets_risk_contribution


def _risk_budget_objective_error(weights, args):

    # The covariance matrix occupies the first position in the variable
    covariances = args[0]

    # The desired contribution of each asset to the portfolio risk occupies the
    # second position
    assets_risk_budget = args[1]

    # We convert the weights to a matrix
    weights = np.matrix(weights)

    # We calculate the portfolio risk (total portfolio variance)
    portfolio_risk = _allocation_risk(weights, covariances)

    # We calculate the contribution of each asset to the risk of the portfolio 
    assets_risk_contribution = _assets_risk_contribution_to_allocation_risk(weights, covariances)

    # We calculate the desired contribution of each asset to the risk of the portfolio
    assets_risk_target = np.asmatrix(np.multiply(portfolio_risk, assets_risk_budget))

    # Error between the desired contribution and the calculated/actual contribution of
    # each asset
    error = sum(np.square(assets_risk_contribution - assets_risk_target.T))[0, 0]
    
    # It returns the calculated error
    return error


def _get_risk_parity_weights(covariances, assets_risk_budget, initial_weights):

    # Restrictions to consider in the optimisation: only long positions whose
    # sum equals 100%
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
                   {'type': 'ineq', 'fun': lambda x: x})

    # Optimisation process in scipy
    optimize_result = minimize(fun=_risk_budget_objective_error,
                               x0=initial_weights,
                               args=[covariances, assets_risk_budget],
                               method='SLSQP',
                               constraints=constraints,
                               tol=TOLERANCE,
                               options={'disp': False})

    # Recover the weights from the optimised object
    weights = optimize_result.x

    # It returns the optimised weights
    return weights


def get_weights(df,portfolio):

    # We download the prices from Yahoo Finance
    prices = portfolio

    # We calculate the covariance matrix
    #covariances = 252.0 * prices.asfreq('W-FRI').pct_change().iloc[1:, :].cov().values
    covariances=np.matrix(df.cov())

    # The desired contribution of each asset to the portfolio risk: we want all
    # asset to contribute equally
    assets_risk_budget = [1 / prices.shape[1]] * prices.shape[1]

    # Initial weights: equally weighted
    #init_weights = [1 / prices.shape[1]] * prices.shape[1]
    init_weights=[]
    sum=0
    for i in range(len(portfolio.columns)-1):
        init_weights.append(0.1)
        sum+=0.1
    init_weights.append(1-sum)
    # Optimisation process of weights
    weights =  _get_risk_parity_weights(covariances, assets_risk_budget, init_weights)

    # Convert the weights to a pandas Series
    weights = pd.Series(weights, index=prices.columns, name='weight')

    # It returns the optimised weights
    return weights


In [161]:
results_return3=pd.DataFrame(columns=stocks)

In [210]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=stocks[1:]
stock=stocks[0:20]+stocks[21:]
portfolio=combine_stocks(stock)   
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=get_weights(df,portfolio)
returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
results_return3[results_return3.columns[20]]=np.array(result) 

                   UNH
Date                  
2022-04-19  535.789307
2022-04-20  544.069763
2022-04-21  535.530212
2022-04-22  519.088867
2022-04-25  522.407043
                    GS
Date                  
2022-04-19  333.905151
2022-04-20  338.984039
2022-04-21  332.265167
2022-04-22  317.823608
2022-04-25  319.463593
                    HD
Date                  
2022-04-19  305.868286
2022-04-20  313.162262
2022-04-21  308.412231
2022-04-22  298.226532
2022-04-25  303.026245
                  MSFT
Date                  
2022-04-19  284.034149
2022-04-20  285.089447
2022-04-21  279.564056
2022-04-22  272.814178
2022-04-25  279.474487
                   MCD
Date                  
2022-04-19  253.992325
2022-04-20  254.439850
2022-04-21  253.843155
2022-04-22  248.791153
2022-04-25  251.486221
                  AMGN
Date                  
2022-04-19  249.424332
2022-04-20  251.403030
2022-04-21  251.481796
2022-04-22  246.451370
2022-04-25  248.243027
                   CAT
Date       

In [211]:
results_return3

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,-0.082755,-0.083995,-0.085926,-0.084168,-0.085355,-0.088685,-0.084941,-0.085673,-0.088007,-0.087749,-0.083706,-0.088513,-0.087624,-0.083728,-0.087569,-0.084023,-0.087148,-0.078876,-0.084871,-0.072368,-0.082986,-0.082396,-0.081308,-0.093728,-0.085048,-0.090182,-0.085097,-0.082905,-0.084629,-0.084747
1,0.066396,0.066113,0.065971,0.065863,0.066741,0.070414,0.066336,0.066209,0.066833,0.069122,0.065612,0.067824,0.066510,0.066249,0.066319,0.065849,0.066562,0.066313,0.066875,0.068531,0.066342,0.065739,0.066769,0.068335,0.066427,0.067262,0.067587,0.065850,0.065989,0.065774
2,-1.695206,-1.721212,-1.754182,-1.730382,-1.725398,-1.682693,-1.729690,-1.744056,-1.762713,-1.700615,-1.729951,-1.744418,-1.765505,-1.713661,-1.769779,-1.728533,-1.756984,-1.638844,-1.714717,-1.490831,-1.700056,-1.706687,-1.664065,-1.807684,-1.728953,-1.783800,-1.699982,-1.711543,-1.734069,-1.741524


In [212]:
results_return3.to_csv("DIAdeleteriskmonth.csv", index=True)

evaluate DIA risk-parity effect using stock swapping with 10 year treasury

In [219]:
results_return4=pd.DataFrame(columns=stocks)

In [253]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=['^TNX']+stocks[1:]
stock=stocks[0:4]+['^TNX']+stocks[5:]
portfolio=combine_stocks(stock)  
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=get_weights(df,portfolio)
returns=portfolio_returns(df,cleaned_weights,20)
sd=portfolio_sd(df,cleaned_weights,20)
sharpe=sharpe_fun(df,cleaned_weights,20,0.0298)
result=[returns,sd,sharpe]
results_return4[results_return4.columns[4]]=np.array(result) 

                   UNH
Date                  
2022-04-19  537.700012
2022-04-20  546.010010
2022-04-21  537.440002
2022-04-22  520.940002
2022-04-25  524.270020
                    GS
Date                  
2022-04-19  335.950012
2022-04-20  341.059998
2022-04-21  334.299988
2022-04-22  319.769989
2022-04-25  321.420013
                    HD
Date                  
2022-04-19  307.799988
2022-04-20  315.140015
2022-04-21  310.359985
2022-04-22  300.109985
2022-04-25  304.940002
                  MSFT
Date                  
2022-04-19  285.299988
2022-04-20  286.359985
2022-04-21  280.809998
2022-04-22  274.029999
2022-04-25  280.720001
             ^TNX
Date             
2022-04-19  2.913
2022-04-20  2.840
2022-04-21  2.917
2022-04-22  2.906
2022-04-25  2.826
                  AMGN
Date                  
2022-04-19  253.369995
2022-04-20  255.380005
2022-04-21  255.460007
2022-04-22  250.350006
2022-04-25  252.169998
                   CAT
Date                  
2022-04-19  232.160004


In [254]:
results_return4

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,-0.075483,-0.076201,-0.076642,-0.073403,-0.077521,-0.079495,-0.076209,-0.077643,-0.079147,-0.080199,-0.076322,-0.079920,-0.079072,-0.075722,-0.080335,-0.076150,-0.079955,-0.071621,-0.077014,-0.063881,-0.077455,-0.073040,-0.071392,-0.086084,-0.078590,-0.081942,-0.074714,-0.075792,-0.077642,-0.075831
1,0.060728,0.060388,0.060165,0.058651,0.061070,0.064275,0.060570,0.060484,0.061142,0.063297,0.060079,0.062019,0.060778,0.060517,0.060809,0.060161,0.060979,0.060582,0.060932,0.061193,0.060856,0.058526,0.059250,0.062533,0.060860,0.061352,0.060168,0.060121,0.060447,0.060069
2,-1.733671,-1.755323,-1.769156,-1.759629,-1.757340,-1.700442,-1.750200,-1.776393,-1.781879,-1.737815,-1.766388,-1.769117,-1.791308,-1.743673,-1.811162,-1.761103,-1.799893,-1.674117,-1.752989,-1.530907,-1.762421,-1.757161,-1.707873,-1.853151,-1.780959,-1.821326,-1.737037,-1.756334,-1.777456,-1.758479


In [255]:
results_return4.to_csv("DIAswapriskmonth.csv", index=True)

quarterly

In [256]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2021,12,19)
end = datetime.datetime(2022,3,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

mean-variance by getting rid of one stock

In [257]:
results_return5=pd.DataFrame(columns=stocks)

In [291]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=stocks[1:]
stock=stocks[0:29]+stocks[30:]
portfolio=combine_stocks(stock)  
# calculate the expected return using historical data
mu = mean_historical_return(portfolio, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=list(cleaned_weights.values())
returns=portfolio_returns(df,cleaned_weights,60)
sd=portfolio_sd(df,cleaned_weights,60)
sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
result=[returns,sd,sharpe]
results_return5[results_return5.columns[29]]=np.array(result) 

                   UNH
Date                  
2021-12-20  483.730011
2021-12-21  487.209991
2021-12-22  494.130005
2021-12-23  495.380005
2021-12-27  499.500000
                    GS
Date                  
2021-12-20  369.432312
2021-12-21  378.091309
2021-12-22  380.089508
2021-12-23  382.783630
2021-12-27  385.766052
                    HD
Date                  
2021-12-20  389.399994
2021-12-21  390.470001
2021-12-22  395.640015
2021-12-23  397.070007
2021-12-27  404.089996
                  MSFT
Date                  
2021-12-20  317.833405
2021-12-21  325.165497
2021-12-22  331.037170
2021-12-23  332.517456
2021-12-27  340.227112
                   MCD
Date                  
2021-12-20  258.847504
2021-12-21  261.478210
2021-12-22  262.022125
2021-12-23  263.021027
2021-12-27  265.285767
                  AMGN
Date                  
2021-12-20  219.990005
2021-12-21  220.919998
2021-12-22  221.529999
2021-12-23  223.789993
2021-12-27  225.169998
                   CAT
Date       

In [292]:
results_return5

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.239355,0.239856,0.239240,0.239749,0.239464,0.239335,0.239149,0.241089,0.239948,0.240036,0.239466,0.240385,0.157273,0.233591,0.239564,0.239740,0.239445,0.240387,0.239399,0.252468,0.239767,0.239944,0.239638,0.239369,0.247075,0.239328,0.238914,0.238865,0.238819,0.240182
1,0.071504,0.071670,0.071466,0.071634,0.071540,0.071497,0.072214,0.073650,0.072478,0.072204,0.071622,0.074208,0.088072,0.071369,0.072441,0.071713,0.072402,0.072320,0.072387,0.078297,0.072509,0.072567,0.071598,0.071509,0.075437,0.071495,0.071731,0.071797,0.071781,0.072163
2,2.930688,2.930883,2.930639,2.930847,2.930729,2.930674,2.898984,2.868843,2.899469,2.911708,2.927401,2.837758,1.447372,2.855475,2.895631,2.927507,2.895559,2.911879,2.895529,2.843902,2.895747,2.895855,2.930795,2.930690,2.880229,2.930673,2.915247,2.911895,2.911884,2.915395


In [293]:
results_return5.to_csv("DIAdeletemeanyear.csv", index=True)

In [359]:
portfolio=combine_stocks(stocks)

                   UNH
Date                  
2021-12-20  480.588898
2021-12-21  484.046295
2021-12-22  490.921387
2021-12-23  492.163239
2021-12-27  496.256500
                    GS
Date                  
2021-12-20  367.183655
2021-12-21  375.789948
2021-12-22  377.775970
2021-12-23  380.453705
2021-12-27  383.417969
                    HD
Date                  
2021-12-20  384.635010
2021-12-21  385.691895
2021-12-22  390.798645
2021-12-23  392.211151
2021-12-27  399.145233
                  MSFT
Date                  
2021-12-20  317.833405
2021-12-21  325.165497
2021-12-22  331.037170
2021-12-23  332.517456
2021-12-27  340.227112
                   MCD
Date                  
2021-12-20  258.847504
2021-12-21  261.478210
2021-12-22  262.022125
2021-12-23  263.021027
2021-12-27  265.285767
                  AMGN
Date                  
2021-12-20  214.723099
2021-12-21  215.630798
2021-12-22  216.226212
2021-12-23  218.432098
2021-12-27  219.779068
                   CAT
Date       

In [294]:
results_return6=pd.DataFrame(columns=stocks)

In [360]:
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC",'^TNX']
portfolio=combine_stocks(stocks)

                   UNH
Date                  
2021-12-20  480.588898
2021-12-21  484.046295
2021-12-22  490.921356
2021-12-23  492.163239
2021-12-27  496.256500
                    GS
Date                  
2021-12-20  367.183655
2021-12-21  375.789948
2021-12-22  377.775970
2021-12-23  380.453705
2021-12-27  383.417969
                    HD
Date                  
2021-12-20  384.635010
2021-12-21  385.691895
2021-12-22  390.798645
2021-12-23  392.211151
2021-12-27  399.145233
                  MSFT
Date                  
2021-12-20  317.833405
2021-12-21  325.165497
2021-12-22  331.037170
2021-12-23  332.517456
2021-12-27  340.227112
                   MCD
Date                  
2021-12-20  258.847504
2021-12-21  261.478210
2021-12-22  262.022125
2021-12-23  263.021027
2021-12-27  265.285767
                  AMGN
Date                  
2021-12-20  214.723068
2021-12-21  215.630798
2021-12-22  216.226212
2021-12-23  218.432098
2021-12-27  219.779053
                   CAT
Date       

In [390]:
#results_volatility=pd.DataFrame(columns=stocks)
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
#stock=stocks[1:]
stock=stocks[0:29]+stocks[30:]
portfolio1=portfolio[stock]  
# calculate the expected return using historical data
mu = mean_historical_return(portfolio, returns_data=False)
# calculate the covariance matrix
S = CovarianceShrinkage(portfolio).ledoit_wolf()
# generate the efficient frontier
ef = EfficientFrontier(mu,S)
# maximize sharpe ratio to help us get the portfolio weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()  
df=pd.DataFrame(columns=portfolio.columns)
for j in df:
    df[j]=portfolio[j].pct_change()
df=df.dropna()
cleaned_weights=list(cleaned_weights.values())
returns=portfolio_returns(df,cleaned_weights,60)
sd=portfolio_sd(df,cleaned_weights,60)
sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
result=[returns,sd,sharpe]
results_return6[results_return6.columns[29]]=np.array(result) 

In [401]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return6[results_return6.columns[i]]=np.array(result) 

In [332]:
result

[0.08498629527217097, 0.04150730728347753, 1.3295561404470706]

In [402]:
results_return6

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.239355,0.239856,0.239240,0.239749,0.239464,0.239328,0.238838,0.240763,0.239618,0.239370,0.239619,0.240621,0.157270,0.234167,0.239456,0.239881,0.239333,0.239711,0.239301,0.253367,0.239673,0.239808,0.239638,0.239369,0.247075,0.239328,0.239388,0.239543,0.239468,0.239301
1,0.071504,0.071670,0.071466,0.071634,0.071540,0.071495,0.071333,0.072633,0.071591,0.071509,0.071591,0.074194,0.088069,0.070525,0.071537,0.071678,0.071497,0.071622,0.071486,0.077221,0.071609,0.071654,0.071598,0.071509,0.075437,0.071495,0.071515,0.071566,0.071542,0.071486
2,2.930687,2.930884,2.930638,2.930846,2.930730,2.930673,2.930442,2.904531,2.930789,2.930696,2.930785,2.841482,1.447386,2.897807,2.930729,2.930884,2.930664,2.930816,2.930661,2.895158,2.930816,2.930861,2.930795,2.930690,2.880228,2.930673,2.930700,2.930757,2.930722,2.930661


In [403]:
results_return6.to_csv("DIAswapmeanquarter.csv", index=True)

test risk parity for quarter and by deletion of stock

In [336]:
results_return7=pd.DataFrame(columns=stocks)

In [393]:
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
portfolio2=combine_stocks(stocks)

                   UNH
Date                  
2021-12-20  480.588898
2021-12-21  484.046295
2021-12-22  490.921387
2021-12-23  492.163239
2021-12-27  496.256500
                    GS
Date                  
2021-12-20  367.183655
2021-12-21  375.789948
2021-12-22  377.775970
2021-12-23  380.453705
2021-12-27  383.417969
                    HD
Date                  
2021-12-20  384.634979
2021-12-21  385.691895
2021-12-22  390.798645
2021-12-23  392.211151
2021-12-27  399.145233
                  MSFT
Date                  
2021-12-20  317.833405
2021-12-21  325.165497
2021-12-22  331.037140
2021-12-23  332.517456
2021-12-27  340.227112
                   MCD
Date                  
2021-12-20  258.847504
2021-12-21  261.478210
2021-12-22  262.022125
2021-12-23  263.021027
2021-12-27  265.285767
                  AMGN
Date                  
2021-12-20  214.723083
2021-12-21  215.630814
2021-12-22  216.226212
2021-12-23  218.432098
2021-12-27  219.779053
                   CAT
Date       

In [394]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return7[results_return7.columns[i]]=np.array(result) 

In [395]:
results_return7

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.023566,0.026917,0.029751,0.026384,0.029163,0.022273,0.019495,0.024064,0.025863,0.023470,0.026590,0.018869,0.011019,0.022439,0.024592,0.025781,0.031319,0.025071,0.027847,0.023398,0.027728,0.028679,0.026344,0.023447,0.023675,0.019206,0.026921,0.027906,0.026581,0.022720
1,0.068144,0.067405,0.068034,0.067638,0.068878,0.068613,0.069179,0.067551,0.067427,0.069347,0.067285,0.068458,0.071560,0.067139,0.068754,0.067380,0.067952,0.067142,0.068920,0.070450,0.067438,0.067399,0.067888,0.069827,0.069313,0.067726,0.070027,0.067785,0.068076,0.070619
2,-0.091484,-0.042771,-0.000724,-0.050509,-0.009255,-0.109709,-0.148958,-0.084914,-0.058389,-0.091286,-0.047703,-0.159679,-0.262448,-0.109643,-0.075747,-0.059648,0.022348,-0.070435,-0.028342,-0.090872,-0.030721,-0.016636,-0.050902,-0.090977,-0.088371,-0.156427,-0.041117,-0.027943,-0.047279,-0.100252


In [338]:
results_return7

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.007855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.039343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.557779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [396]:
results_return7.to_csv("DIAdeleteriskquarter.csv", index=True)

In [400]:
results_return8=pd.DataFrame(columns=stocks)

In [404]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,60)
    sd=portfolio_sd(df,cleaned_weights,60)
    sharpe=sharpe_fun(df,cleaned_weights,60,0.0298)
    result=[returns,sd,sharpe]
    results_return8[results_return8.columns[i]]=np.array(result) 

In [405]:
results_return8

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.023566,0.026917,0.029751,0.026384,0.029163,0.022273,0.019495,0.024064,0.025863,0.023470,0.026590,0.018869,0.011019,0.022439,0.024592,0.025781,0.031319,0.025071,0.027847,0.023398,0.027728,0.028679,0.026344,0.023447,0.023675,0.019206,0.026921,0.027906,0.026581,0.022720
1,0.068144,0.067405,0.068034,0.067638,0.068878,0.068613,0.069179,0.067551,0.067427,0.069347,0.067285,0.068458,0.071560,0.067139,0.068754,0.067380,0.067952,0.067142,0.068920,0.070450,0.067438,0.067399,0.067888,0.069827,0.069313,0.067726,0.070027,0.067785,0.068076,0.070619
2,-0.091485,-0.042771,-0.000724,-0.050510,-0.009255,-0.109709,-0.148958,-0.084914,-0.058389,-0.091286,-0.047704,-0.159679,-0.262448,-0.109643,-0.075747,-0.059648,0.022348,-0.070435,-0.028342,-0.090872,-0.030722,-0.016636,-0.050902,-0.090977,-0.088371,-0.156427,-0.041117,-0.027943,-0.047279,-0.100252


In [406]:
results_return8.to_csv("DIAswapriskquarter.csv", index=True)

for yearly data!

In [482]:
from functools import reduce
# we plan to evaluate the 1-month previous data 
start = datetime.datetime(2020,6,19)
end = datetime.datetime(2021,6,19)
# a function to read a single stock
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 
# a function to read different stocks and combine them into a single table
def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    #print(df_merged.head())
    return df_merged

In [483]:
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC",'^TNX']
portfolio=combine_stocks(stocks)

                   UNH
Date                  
2020-06-19  283.147247
2020-06-22  284.537537
2020-06-23  289.330536
2020-06-24  281.144501
2020-06-25  287.988861
                    GS
Date                  
2020-06-19  193.557953
2020-06-22  195.276291
2020-06-23  196.581848
2020-06-24  190.092499
2020-06-25  198.808975
                    HD
Date                  
2020-06-19  236.106979
2020-06-22  238.509735
2020-06-23  239.610550
2020-06-24  235.609222
2020-06-25  234.891281
                  MSFT
Date                  
2020-06-19  191.307037
2020-06-22  196.620331
2020-06-23  197.933914
2020-06-24  193.944107
2020-06-25  196.394836
                   MCD
Date                  
2020-06-19  178.266800
2020-06-22  179.126831
2020-06-23  178.324127
2020-06-24  176.097733
2020-06-25  174.635727
                  AMGN
Date                  
2020-06-19  222.975250
2020-06-22  218.762360
2020-06-23  220.219589
2020-06-24  217.193008
2020-06-25  219.621750
                   CAT
Date       

In [484]:
stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
portfolio2=combine_stocks(stocks)

                   UNH
Date                  
2020-06-19  283.147217
2020-06-22  284.537506
2020-06-23  289.330536
2020-06-24  281.144501
2020-06-25  287.988831
                    GS
Date                  
2020-06-19  193.557953
2020-06-22  195.276306
2020-06-23  196.581833
2020-06-24  190.092484
2020-06-25  198.808990
                    HD
Date                  
2020-06-19  236.106995
2020-06-22  238.509720
2020-06-23  239.610550
2020-06-24  235.609207
2020-06-25  234.891296
                  MSFT
Date                  
2020-06-19  191.307037
2020-06-22  196.620346
2020-06-23  197.933929
2020-06-24  193.944092
2020-06-25  196.394867
                   MCD
Date                  
2020-06-19  178.266815
2020-06-22  179.126801
2020-06-23  178.324112
2020-06-24  176.097717
2020-06-25  174.635757
                  AMGN
Date                  
2020-06-19  222.975266
2020-06-22  218.762360
2020-06-23  220.219604
2020-06-24  217.193024
2020-06-25  219.621750
                   CAT
Date       

In [410]:
results_return9=pd.DataFrame(columns=stocks)

delete for mean variance

In [413]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return9[results_return9.columns[i]]=np.array(result) 

In [414]:
results_return9

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.497421,0.451425,0.487520,0.491188,0.511987,0.487522,0.466569,0.487530,0.487552,0.487533,0.487449,0.487536,0.487530,0.487456,0.487535,0.468017,0.487540,0.487528,0.487522,0.487524,0.487521,0.487515,0.487204,0.487534,0.487530,0.487550,0.487532,0.487527,0.487518,0.487514
1,0.188852,0.174229,0.182898,0.184643,0.194156,0.182899,0.183786,0.182900,0.182905,0.182900,0.182880,0.182901,0.182900,0.182882,0.182901,0.179667,0.182903,0.182900,0.182897,0.182899,0.182898,0.182896,0.183790,0.182902,0.182900,0.182905,0.182900,0.182899,0.182897,0.182897
2,2.476127,2.419939,2.502602,2.498813,2.483502,2.502602,2.376507,2.502625,2.502673,2.502635,2.502451,2.502647,2.502625,2.502466,2.502635,2.439047,2.502638,2.502619,2.502622,2.502611,2.502598,2.502598,2.488733,2.502623,2.502627,2.502670,2.502635,2.502623,2.502595,2.502578


In [415]:
results_return9.to_csv("DIAdeletemeanyear.csv", index=True)

swap for the mean variance

In [416]:
results_return10=pd.DataFrame(columns=stocks)

In [418]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    # calculate the expected return using historical data
    mu = mean_historical_return(portfolio1, returns_data=False)
    # calculate the covariance matrix
    S = CovarianceShrinkage(portfolio1).ledoit_wolf()
    # generate the efficient frontier
    ef = EfficientFrontier(mu,S)
    # maximize sharpe ratio to help us get the portfolio weights
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()  
    cleaned_weights=list(cleaned_weights.values())
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return10[results_return10.columns[i]]=np.array(result) 

In [419]:
results_return10.to_csv("DIAswapmeanyear.csv", index=True)

test for risk-parity

delete testing

In [485]:
results_return11=pd.DataFrame(columns=stocks)

In [486]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC"]
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio2[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return11[results_return11.columns[i]]=np.array(result) 

In [422]:
results_return11

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC,^TNX
0,0.064993,0.063470,0.065837,0.064801,0.065947,0.067683,0.063671,0.066127,0.064638,0.066808,0.064757,0.065714,0.067457,0.065704,0.065813,0.063480,0.065946,0.064749,0.067041,0.066867,0.064388,0.065063,0.064326,0.067645,0.066286,0.065752,0.068030,0.066210,0.065855,0.067357,NaN
1,0.069375,0.068620,0.069786,0.068913,0.070249,0.069592,0.068746,0.068650,0.068399,0.070045,0.069117,0.068804,0.070698,0.070572,0.068957,0.068674,0.069611,0.067878,0.070560,0.070643,0.068637,0.069511,0.068896,0.070387,0.069353,0.070565,0.070595,0.069182,0.068748,0.068983,NaN
2,0.507287,0.490676,0.516391,0.507905,0.514555,0.544366,0.492701,0.529158,0.509334,0.528343,0.505760,0.521976,0.532646,0.508756,0.522255,0.490430,0.519259,0.514881,0.527801,0.524717,0.503925,0.507304,0.501132,0.537673,0.526086,0.509488,0.541530,0.526289,0.524455,0.544432,NaN


In [487]:
results_return11.to_csv("DIAdeleteriskyear.csv", index=True)

swap testings

In [488]:
results_return12=pd.DataFrame(columns=stocks)

In [489]:
for i in range(30):
    #results_volatility=pd.DataFrame(columns=stocks)
    stocks = ["UNH", "GS", "HD","MSFT","MCD","AMGN","CAT","V","HON","JNJ","CRM","TRV","CVX","AXP","IBM","AAPL","MMM","BA","PG","WMT","JPM","NKE","DIS","MRK","KO","DOW","VZ","CSCO","WBA","INTC",'^TNX']
    #stock=stocks[1:]
    if i!=0:
        stock=stocks[0:i]+stocks[i+1:]
    else:
        stock=stocks[1:]
    portfolio1=portfolio[stock]  
    df=pd.DataFrame(columns=portfolio1.columns)
    for j in df:
        df[j]=portfolio1[j].pct_change()
    df=df.dropna()
    cleaned_weights=get_weights(df,portfolio1)

    returns=portfolio_returns(df,cleaned_weights,253)
    sd=portfolio_sd(df,cleaned_weights,253)
    sharpe=sharpe_fun(df,cleaned_weights,253,0.0298)
    result=[returns,sd,sharpe]
    results_return12[results_return12.columns[i]]=np.array(result) 

In [490]:
results_return12

,UNH,GS,HD,MSFT,MCD,AMGN,CAT,V,HON,JNJ,CRM,TRV,CVX,AXP,IBM,AAPL,MMM,BA,PG,WMT,JPM,NKE,DIS,MRK,KO,DOW,VZ,CSCO,WBA,INTC
0,0.288533,0.281725,0.291770,0.287059,0.292366,0.299298,0.282761,0.292658,0.286559,0.295771,0.280242,0.291187,0.291712,0.285539,0.292017,0.281332,0.292160,0.287635,0.296804,0.296106,0.285619,0.288605,0.285206,0.299080,0.293556,0.285654,0.300539,0.293462,0.291866,0.298783
1,0.144399,0.142394,0.145311,0.143524,0.145943,0.144731,0.142642,0.142694,0.142094,0.145530,0.141224,0.142822,0.142019,0.141697,0.143481,0.143044,0.144563,0.140733,0.146646,0.146720,0.142272,0.144437,0.143008,0.146149,0.144104,0.141767,0.146520,0.143986,0.142828,0.142802
2,1.791789,1.769207,1.802821,1.792447,1.799104,1.862064,1.773397,1.842118,1.806962,1.827609,1.773362,1.830154,1.844201,1.804833,1.827541,1.758419,1.814854,1.832078,1.820743,1.815063,1.798094,1.791826,1.785961,1.842506,1.830315,1.804750,1.847796,1.831163,1.834840,1.883612


In [491]:
results_return12.to_csv("DIAswapriskyear.csv", index=True)